In [1]:
# set this to your working directory
WORK_DIR = '/home/elliott/Dropbox/_Ash_Teaching/2018-09 - Bocconi - Text Data and ML/code'
import os
os.chdir(WORK_DIR)
import pandas as pd

X = pd.read_pickle('X.pkl').toarray()
vocab = pd.read_pickle('vocab.pkl')
df1 = pd.read_csv('death-penalty-cases.csv')
Y = df1['citeCount'] > 0
num_features = X.shape[1]

In [2]:
# Set up the basic model
from keras.models import Sequential
from keras.layers import Activation, Dense
model = Sequential()
model.add(Dense(64, input_dim=num_features, activation='relu')) 

Using TensorFlow backend.


In [3]:
# initializers
model.add(Dense(64, kernel_initializer='he_normal'))
model.add(Dense(64, kernel_initializer='he_uniform'))

In [4]:
# other activation functions (https://keras.io/activations/)
model.add(Dense(64, activation="elu"))

In [5]:
# batch normalization
from keras.layers.normalization import BatchNormalization
model.add(Dense(64, use_bias=False)) 
model.add(BatchNormalization())
model.add(Activation('relu'))

In [6]:
# regularization
from keras.regularizers import l1, l2, l1_l2
model.add(Dense(64, 
                kernel_regularizer=l2(0.01),
                activity_regularizer=l1(0.01)))
model.add(Dense(64, 
                kernel_regularizer=l1_l2(l1=0.01,l2=.01),
                activity_regularizer=l1_l2(l1=0.01,l2=.01)))

In [7]:
# Dropout
from keras.layers import Dropout
model.add(Dropout(0.5))

In [8]:
model.add(Dense(1,activation='sigmoid'))

In [9]:
# Optimizers
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
# Early stopping
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', 
                          min_delta=0.0001, 
                          patience=5, 
                          mode='auto')
model.fit(X, Y, batch_size=128, 
           epochs=100, 
           callbacks=[earlystop], 
           validation_split=0.2)

Train on 26053 samples, validate on 6514 samples
Epoch 1/100
26053/26053 [==============================] - 1s 43us/step - loss: 2.2133 - acc: 0.8207 - val_loss: 1.7595 - val_acc: 0.3753
Epoch 2/100
26053/26053 [==============================] - 1s 41us/step - loss: 2.0302 - acc: 0.8207 - val_loss: 1.6829 - val_acc: 0.3753
Epoch 3/100
26053/26053 [==============================] - 1s 39us/step - loss: 1.9279 - acc: 0.8207 - val_loss: 1.5986 - val_acc: 0.3753
Epoch 4/100
26053/26053 [==============================] - 1s 39us/step - loss: 1.8957 - acc: 0.8207 - val_loss: 1.6613 - val_acc: 0.3753
Epoch 5/100
26053/26053 [==============================] - 1s 41us/step - loss: 1.8356 - acc: 0.8207 - val_loss: 1.6818 - val_acc: 0.3753
Epoch 6/100
26053/26053 [==============================] - 1s 39us/step - loss: 1.7929 - acc: 0.8207 - val_loss: 1.4689 - val_acc: 0.3753


In [12]:
# Batch Training with Large Data
from numpy import memmap
X_mm = memmap('X.pkl',shape=(32567, 472))

model.fit(X_mm, Y, batch_size=128, 
           epochs=3, 
           validation_split=0.2)

Train on 26053 samples, validate on 6514 samples
Epoch 1/3
26053/26053 [==============================] - 1s 38us/step - loss: 3.0958 - acc: 0.8207 - val_loss: 2.0465 - val_acc: 0.3753
Epoch 2/3
26053/26053 [==============================] - 1s 37us/step - loss: 2.2566 - acc: 0.8207 - val_loss: 1.7367 - val_acc: 0.3753
Epoch 3/3
26053/26053 [==============================] - 1s 38us/step - loss: 1.9223 - acc: 0.8207 - val_loss: 1.8935 - val_acc: 0.3753


In [13]:
# Grid search with KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# instantiate KerasClassifier with build function
def create_model(hidden_layers=1):  
    model = Sequential()
    model.add(Dense(16, input_dim=num_features, 
                    activation='relu')) 
    for i in range(hidden_layers):
        model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', 
                optimizer='adam', 
                metrics= ['accuracy'])
    return model
clf = KerasClassifier(create_model)

# set of grid search CV to select number of hidden layers
params = {'hidden_layers' : [0,1,2,3]}
grid = GridSearchCV(clf, param_grid=params)
grid.fit(X,Y)
grid.best_params_

Epoch 1/1
21711/21711 [==============================] - 1s 29us/step
Epoch 1/1
21711/21711 [==============================] - 1s 29us/step
Epoch 1/1
21712/21712 [==============================] - 1s 29us/step
Epoch 1/1
21711/21711 [==============================] - 1s 36us/step
Epoch 1/1
21711/21711 [==============================] - 1s 31us/step
Epoch 1/1
21712/21712 [==============================] - 1s 33us/step
Epoch 1/1
21711/21711 [==============================] - 1s 33us/step
Epoch 1/1
21711/21711 [==============================] - 1s 33us/step
Epoch 1/1
21712/21712 [==============================] - 1s 34us/step
Epoch 1/1
21711/21711 [==============================] - 1s 35us/step
Epoch 1/1
21711/21711 [==============================] - 1s 37us/step
Epoch 1/1
21712/21712 [==============================] - 1s 37us/step
Epoch 1/1
32567/32567 [==============================] - 3s 95us/step - loss: 0.5043 - acc: 0.7630


{'hidden_layers': 0}